In [3]:
# System imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import panel as pn
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact
from panel import widgets
from string import digits
import csv
import json
import numpy as np
import dateparser
import panel as pn

pn.extension()

%matplotlib inline


In [4]:
# Local imports
import sys
sys.path.append("../lib2")

from Constants import Constants
from ProcessingTools import DateProcessingTools


In [5]:
# Construct the tools
debug_level = 0
constants = Constants()
tool_data_processing = DateProcessingTools(debug_level)


In [6]:
# Import all preprocessed data
atlas           = pd.read_pickle(constants.PREPROCESSED_ATLAS_FILE_PATH)
forecast        = pd.read_pickle(constants.PREPROCESSED_FORECAST_DATA_FILE_PATH)
revenue2020     = pd.read_pickle(constants.PREPROCESSED_REVENUE2020_FILE_PATH)
revenue2020A    = pd.read_pickle(constants.PREPROCESSED_REVENUE2020A_FILE_PATH)


In [7]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

In [8]:
# Calculate the mean values for each neighborhood
#rankings_pd.rename(columns = {'test':'TEST', 'odi':'ODI', 
#                              't20':'T20'}, inplace = True) 

#atlas2.columns = ["Customers", "Lat", "Long"]

total_data = atlas.loc[:, ["Number of Users", "Invoice Amount"]].groupby(["Customers"]).sum()
# total_data.dropna(inplace=True)
total_data

atlas_with_totals = pd.concat([atlas.drop(columns=["Number of Users", "Invoice Amount"]), total_data], axis="columns", join="inner")
atlas_with_totals.head(5)


,Invoice Date,Invoice #,Subscription,Account Code,Address,State,Lat,Long,Service Start,Service End,Subscription Duration,Number of Users,Invoice Amount
Customers,,,,,,,,,,,,,
Alabama A&M University-College of Education,2018-03-23 00:00:00-04:00,105,3 Years,4700-0-00-00000-00-0000,"4900 Meridian St N, Huntsville, AL 35811",AL,34.785061,-86.573280,2018-03-21 00:00:00-04:00,2021-03-31 00:00:00-04:00,1095 days,0.0,5460.0
Albertus Magnus College,2018-05-16 00:00:00-04:00,127,3 Years,4700-0-00-00000-00-0000,"700 Prospect St, New Haven, CT 06511",CT,41.331299,-72.921440,2018-09-01 00:00:00-04:00,2021-08-31 00:00:00-04:00,1095 days,0.0,5670.0
Albuquerque Public Schools,2018-11-20 00:00:00-05:00,161,1 Year,4700-0-00-00000-00-0000,"Po Box 25704 Albuquerque, NM 87125",NM,35.092410,-106.642950,2018-11-19 00:00:00-05:00,2019-11-30 00:00:00-05:00,365 days,0.0,2270.0
Alcorn State University,2017-10-31 00:00:00-04:00,NaN,1 Year,4700-0-00-00000-00-0000,"1000 Asu Dr, Lorman, MS 39096",MS,31.875710,-91.141739,2017-08-31 00:00:00-04:00,2018-08-31 00:00:00-04:00,365 days,0.0,1700.0
Alderson Broaddus University,2020-10-07 00:00:00-04:00,NB100820K,3 Months,4700-0-00-00000-00-0000,"101 College Hill Rd, Philippi, WV 26416",WV,39.160261,-80.049048,2020-10-02 00:00:00-04:00,2020-12-31 00:00:00-05:00,90 days,50.0,500.0


In [9]:
px.set_mapbox_access_token(mapbox_token)
map = px.scatter_mapbox(
    atlas_with_totals.reset_index(),
    lat="Lat",
    lon="Long",
    # size="Invoice Amount",
    size="Number of Users",
    color="Customers",
    width=1800,
    height=900,
    zoom=4,
    mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
)
map.show()


In [13]:
# def generate_map(state):
#     px.set_mapbox_access_token(mapbox_token)
#     map = px.scatter_mapbox(
#         atlas_with_totals.iloc[:10].reset_index(),
#         lat="Lat",
#         lon="Long",
#         size="Invoice Amount",
#         # size="Number of Users",
#         color="Customers",
#         width=800,
#         height=400,
#         # width=1800,
#         # height=900,
#         zoom=4,
#         mapbox_style="open-street-map"  # Options: [ "open-street-map, white-bg, carto-positron, carto-darkmatter, stamen-terrain, stamen-toner, stamen-watercolor" ]
#     )
#     return map

# map = generate_map("")
# map.show()



# # Group by year and neighborhood and then create a new dataframe of the mean values
# atlas_with_totals["ServiceStart_Year"] = atlas_with_totals["Service Start"].apply(lambda x: x.year)
# atlas_with_totals_grouped = atlas_with_totals.groupby(["ServiceStart_Year", "State"]).sum()
# atlas_with_totals_grouped.head(5)

# # Use hvplot to create an interactive line chart of the average price per sq ft.
# plt_atlas_with_totals_grouped = atlas_with_totals_grouped.hvplot.line(
#     x="ServiceStart_Year",
#     y="Invoice Amount",
#     xlabel="ServiceStart_Year",
#     ylabel="Invoice Amount",
#     groupby="State",
#     width=600
# ).opts(yformatter="%0f")
# plt_atlas_with_totals_grouped


:Curve   [ServiceStart_Year]   (Invoice Amount)

In [11]:
total_data.hvplot(kind='line', figsize=(20, 10))

:NdOverlay   [Variable]
   :Curve   [Customers]   (value)